In [ ]:
%load_ext autoreload
%autoreload 2

# System
import os
import sys

# File Formatting
import pandas as pd

# Math and Plotting
import numpy as np
import matplotlib.pyplot as plt

# PyTorch
import torch

# Dataset and DataLoader
from datasets import PhysioNetDataset
from torch.utils.data import DataLoader

# Model and Trainer
from torch.nn import DataParallel
from torch.nn import Linear
from models import Trainer

# Loss and Optimizer
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

In [ ]:
root = os.getcwd()
meta = "REFERENCE.csv"
fs = 300 # Hz
seconds = 30
trainset = PhysioNetDataset(root, meta, fs, seconds, train = True)
testset = PhysioNetDataset(root, meta, fs, seconds, train = False)
print("Trainset Sample Size: {:d}\tTestset Sample Size: {:d}".format(len(trainset), len(testset)))

In [ ]:
idx = 1000
img, lbl = trainset[idx]
print(img.shape)
plt.imshow(img[0,:,:])
plt.title("Label: {:d}".format(lbl))

In [ ]:
batch_size = 32
train_loader = DataLoader(trainset, batch_size, shuffle = True)
test_loader = DataLoader(testset, batch_size, shuffle = False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn = torch.hub.load("pytorch/vision:v0.6.0", "resnet18", pretrained = False)
if(1 < torch.cuda.device_count()):
    cnn = DataParallel(cnn)
cnn.to(device)
loss = CrossEntropyLoss()
optim = SGD(cnn.parameters(), lr = 0.001, momentum = 0.90)
trainer = Trainer(cnn, loss, optim)

In [ ]:
trainer.train(train_loader, 200, True)

In [ ]:
trainer.validate(test_loader)